In [ ]:
# ## 1. Install Required Packages 
# !pip install transformers torch pandas tqdm scikit-learn

In [1]:
# ## 2. Import Libraries
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from sklearn.metrics import accuracy_score

C:\Users\yozev\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# ## 3. Load the Dataset
data_path = r"C:\Users\yozev\PycharmProjects\Probing-Slang-Ambiguity-in-LLM\using_claude\manual_slang_dataset.csv"
df = pd.read_csv(data_path).rename(columns={"sentence": "text", "binary": "label"})
print("Total examples:", len(df))
display(df.head())

Total examples: 736


,text,label
0,"That new song is absolutely fire, I can't stop...",1
1,The fire department responded quickly to the h...,0
2,"Your outfit is so bad, everyone's going to be ...",1
3,I got a bad grade on my chemistry test yesterday,0
4,"She killed that performance, the crowd went wild",1


In [6]:
# ## 4. Initialize Your Open-Source LLM (e.g. LLaMA)
model_name_or_path = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16  
)
model.eval()

# Use GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

C:\Users\yozev\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yozev\.cache\huggingface\hub\models--meta-llama--Llama-2-7b-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to reg

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (no

In [7]:
# ## 5. Define a Prompting Function
def classify_with_llm(sentence, tokenizer, model, device, max_new_tokens=32):
    """
    Prompt the LLM to choose between 'slang' or 'literal' for a given sentence.
    Returns exactly 'slang' or 'literal'.
    """
    prompt = (
        "Classify the following sentence as either 'slang' or 'literal'.\n"
        f"Sentence: \"{sentence}\"\n"
        "Answer with exactly one word (slang or literal):"
    )
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Greedy decoding for a short answer
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Discard prompt tokens, decode only the newly generated portion
    generated_tokens = outputs[0][ inputs["input_ids"].shape[-1] : ]
    text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip().lower()

    # Take the first word (in case the model returns extra text)
    first_word = text.split()[0] if text else ""
    # If it’s neither exactly 'slang' nor 'literal', attempt to find the substring
    if first_word not in ["slang", "literal"]:
        if "slang" in first_word:
            first_word = "slang"
        elif "literal" in first_word:
            first_word = "literal"
        else:
            first_word = "literal"  # fallback default
    return first_word

In [8]:
# ## 6. Run the Zero-Shot Classification Over the Whole Dataset
predictions = []
true_labels = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Classifying"):
    sentence = row["text"]
    true_label = "slang" if row["label"] == 1 else "literal"
    pred_label = classify_with_llm(sentence, tokenizer, model, device)
    predictions.append(pred_label)
    true_labels.append(true_label)

# ## 7. Compute and Print Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"\nZero-Shot LLaMA Classification Accuracy: {accuracy:.4f}")

# ## 8. Inspect a Few Examples
example_df = pd.DataFrame({
    "sentence": df["text"],
    "true": true_labels,
    "predicted": predictions
})
display(example_df.head(10))

Classifying:   0%|          | 0/736 [00:00<?, ?it/s]C:\Users\yozev\AppData\Roaming\Python\Python311\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Classifying:   1%|▏         | 10/736 [06:22<7:43:22, 38.29s/it]


KeyboardInterrupt: 